## Extracting OpenStreetMap data for Vancouver cities

This notebook uses the [OpenStreetMap Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API) to extract the data representing each city that makes up Metro Vancouver.

The original query was done on OverPass Turbo: https://overpass-turbo.eu/s/2idu

This notebook uses a Python interpreter that runs using WebAssembly inside the browser (Pyiodide inside Jupyter Lite). Because the whole website can be rendered out as a static site, it can easily be hosted from IPFS, and published to a website location that logically corresponds to a physical location in the Vancouver area. See https://hex.camp/ for more info.

The website is built and published using GitHub Actions here: https://github.com/hexcamp/vanhex-openstreetmap-notebooks

In [1]:
query = """[out:json][timeout:25];
area(id:3602218280)->.searchArea;
relation["boundary"="administrative"]["name"]["name"!="Electoral Area A"]["admin_level"="8"](area.searchArea);
out geom;"""

In [2]:
# Helper for encoding the query for the POST request

from urllib.parse import quote

def encodeURIComponent(s):
    # Matches JavaScript's encodeURIComponent behavior by not encoding
    # A-Z a-z 0-9 - _ . ! ~ * ' ( )
    # All other characters, including special URL delimiters like /, ?, &, #, etc., are encoded.
    return quote(s, safe="~*'()")

In [3]:
# Make the HTTP request to the Overpass API

from pyodide.http import pyfetch
response = await pyfetch("https://overpass-api.de/api/interpreter", method="POST", body="data=" + encodeURIComponent(query))
osm_data = await response.json()

In [4]:
# Convert to GeoJSON format for display

import micropip
await micropip.install("osm2geojson")
import osm2geojson

geojson_data = osm2geojson.json2geojson(osm_data)

In [5]:
# Display the cities on a map

from IPython.display import GeoJSON

In [6]:
GeoJSON(geojson_data)

<IPython.display.GeoJSON object>